In [10]:
import sys
import os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

In [11]:
from pathlib import Path
from src.utils.config_loader import load_config

base_dir = Path(os.getcwd()).parent

config = load_config(base_dir / 'secrets.yaml')

In [12]:
from langchain_openai import AzureChatOpenAI
from langchain_core.messages import (SystemMessage, HumanMessage)

os.environ['OPENAI_API_VERSION'] = config['OPENAI_API_VERSION']
os.environ['AZURE_OPENAI_ENDPOINT'] = config['OPENAI_API_BASE']
os.environ['AZURE_OPENAI_API_KEY'] = config['OPENAI_API_KEY']

llm = AzureChatOpenAI(
    deployment_name=config['OPENAI_DEPLOYMENT_NAME'],
    temperature=0.0
)

# TODO: Maybe for this task i should experiment with temperature

In [13]:
import json

with open(base_dir / 'data/versioned_text_changes_with_summaries.json', 'r') as f:
    data = json.load(f)


In [14]:
results = []

data_sample = data[:1]

for item in data_sample:

    version_1 = item['version_1']
    version_2 = item['version_2']
    ref_summary = item['ref_summary']

    msg = [
    SystemMessage(
        content=(
            "You are a helpful assistant that summarizes the differences "
            "between two versions of a text."
        )
    ),
        HumanMessage(
            content=f"""\
    Please summarize the differences between the following two versions of a text:

    Version 1:
    {version_1}

    Version 2:
    {version_2}
    """
        )
    ]

    try: 
        response = llm.invoke(msg)

        results.append({
            'id': item['id'],
            'ref_summary': ref_summary,
            'model_summary': response.content
        })

    except Exception as e:
        print(f"Error: {e}")
        continue

In [15]:
with open(base_dir / f"results/{config['OPENAI_DEPLOYMENT_NAME']}_SHOT.jsonl", "w") as f:
    for result in results:
        json.dump(result, f, ensure_ascii=False)
        f.write('\n')